In [1]:
import pandas as pd
import os

In [2]:
fut = pd.read_pickle("C:\\PICKLE\\Nifty Future\\2019-02-11_nifty_future.pkl").set_index("date_time")
fut.index = pd.to_datetime(fut.index)
opt = pd.read_pickle("C:\\PICKLE\\Nifty Options\\2019-02-11_nifty.pkl").set_index("date_time")
opt.index = pd.to_datetime(opt.index)

In [3]:
fut.head(3)

,scrip,open,high,low,close,volume,openinterest
date_time,,,,,,,
2019-02-11 09:15:00,NIFTY-I,10920.0,10929.90,10915.60,10917.40,116625,20999550
2019-02-11 09:16:00,NIFTY-I,10918.0,10918.85,10901.00,10909.55,163125,21024375
2019-02-11 09:17:00,NIFTY-I,10908.7,10909.70,10904.15,10908.90,120900,21024375


In [4]:
entry_time = pd.to_datetime("2019-02-11 09:20:00")
exit_time = pd.to_datetime("2019-02-11 15:00:00")
sl = 20
dk = 20

In [5]:
valid_time = fut.loc[entry_time:exit_time].index

In [6]:
valid_time

DatetimeIndex(['2019-02-11 09:20:00', '2019-02-11 09:21:00',
               '2019-02-11 09:22:00', '2019-02-11 09:23:00',
               '2019-02-11 09:24:00', '2019-02-11 09:25:00',
               '2019-02-11 09:26:00', '2019-02-11 09:27:00',
               '2019-02-11 09:28:00', '2019-02-11 09:29:00',
               ...
               '2019-02-11 14:51:00', '2019-02-11 14:52:00',
               '2019-02-11 14:53:00', '2019-02-11 14:54:00',
               '2019-02-11 14:55:00', '2019-02-11 14:56:00',
               '2019-02-11 14:57:00', '2019-02-11 14:58:00',
               '2019-02-11 14:59:00', '2019-02-11 15:00:00'],
              dtype='datetime64[ns]', name='date_time', length=341, freq=None)

In [28]:
from tqdm import tqdm

In [49]:
sd ,SDroundoff = 0,False
for current_dt in valid_time:
    future_price = fut.loc[current_dt,'close']
    round_price = round(future_price/50)*50

    ce_scrip,pe_scrip = f"{round_price}CE",f"{round_price}PE"
    # print(ce_scrip,pe_scrip)
    # ce_price,pe_price = opt.loc[(current_dt,ce_scrip),'close'] , opt.loc[(current_dt,pe_scrip),'close']
    ce_price,pe_price = opt[opt.scrip == ce_scrip].loc[current_dt,'close'] , opt[opt.scrip == ce_scrip].loc[current_dt,'close']
    # print(ce_price,pe_price)

    syn_future = ce_price-pe_price + round_price
    round_syn_future=round(syn_future/50)*50

    ce_scrip_list = [f"{round_syn_future}CE", f"{round_syn_future+50}CE", f"{round_syn_future-50}CE"]
    pe_scrip_list = [f"{round_syn_future}PE", f"{round_syn_future+50}PE", f"{round_syn_future-50}PE"]
                
    # print(ce_scrip_list,pe_scrip_list)
    scrip_index,min_val = None , float("inf")

    for i in range(3):
        try:
            ce_price = opt[opt.scrip == ce_scrip_list[i]].loc[current_dt,'close']
            pe_price = opt[opt.scrip == pe_scrip_list[i]].loc[current_dt,'close']
            diff = abs(ce_price - pe_price)
            if min_val> diff:
                min_val = diff
                scrip_index = i 

            # print(opt[opt.scrip == ce_scrip_list[scrip_index]].loc[valid_time[0],'close'])
        except:
            pass
    ce_scrip,pe_scrip = ce_scrip_list[scrip_index],pe_scrip_list[scrip_index]
    ce_price ,pe_price = opt[opt.scrip == ce_scrip].loc[current_dt,'close'], opt[opt.scrip == pe_scrip].loc[current_dt,'close']
    print(ce_price,pe_price)

    if sd:
        sd_range = (ce_price+pe_price)*sd
        if SDroundoff:
            sd_range = round(sd_range/50)*50
        else:
            sd_range = max(50,round(sd_range/50)*50)

        ce_scrip,pe_scrip = f"{int(ce_scrip[:-2])+sd_range}CE",f"{int(pe_scrip[:-2])+sd_range}PE"
        ce_price,pe_price = opt[opt.scrip == ce_scrip].loc[valid_time[0],'close'] , opt[opt.scrip == pe_scrip].loc[valid_time[0],'close']





55.55 69.7
58.65 66.7
59.55 67.0
59.0 66.45
59.2 65.15
57.95 66.9
57.15 67.2
55.55 69.95
53.6 73.05
53.25 75.55
78.45 54.45
53.85 74.25
54.65 73.15
55.45 70.1
55.0 69.25
55.15 69.55
54.6 70.75
52.7 70.5
54.25 68.45
55.15 66.7
56.0 64.0
55.45 65.35
54.0 67.8
53.7 68.1
53.95 67.8
51.8 69.95
52.1 71.1
50.6 73.05
75.0 52.75
73.45 56.1
75.5 52.5
72.65 54.35
73.3 54.15
69.5 54.95
69.5 55.2
73.1 51.65
70.35 56.0
68.75 56.0
70.0 55.55
72.5 53.15
72.25 53.0
72.95 53.0
73.45 51.9
74.0 51.1
72.75 52.85
73.15 52.95
49.8 74.55
73.75 50.8
73.1 51.4
73.4 51.25
72.4 53.6
72.4 53.6
74.0 50.55
74.3 50.7
49.8 73.0
51.2 71.4
51.9 69.1
50.7 69.75
52.2 68.85
51.5 69.55
50.8 70.9
50.9 70.65
50.05 71.85
50.2 71.9
51.4 70.0
50.5 71.65
49.3 72.65
50.3 72.0
50.75 71.3
50.9 71.45
50.7 71.5
75.5 51.05
75.0 50.4
74.6 51.5
73.6 52.7
74.95 51.2
49.5 73.25
74.0 51.0
73.6 51.65
72.6 52.0
72.45 52.9
75.0 52.15
73.0 53.0
74.0 50.85
49.0 73.7
49.25 73.65
49.45 73.8
49.8 72.1
49.95 72.45
49.6 73.2
74.35 50.4
74.1 50.7
74.2

In [13]:
ce_scrip,pe_scrip = f"{round_fp}CE",f"{round_fp}PE"

In [20]:
ce_price,pe_price = opt[opt.scrip == ce_scrip].loc[valid_time[0],'close'] , opt[opt.scrip == pe_scrip].loc[valid_time[0],'close']

In [22]:
syn_future = ce_price-pe_price + round_fp
syn_future

np.float64(10885.85)

In [ ]:
round_syn_future=round(syn_future/50)*50

10900

In [33]:
ce_scrip_list = [f"{round_syn_future}CE", f"{round_syn_future+50}CE", f"{round_syn_future-50}CE"]
pe_scrip_list = [f"{round_syn_future}PE", f"{round_syn_future+50}PE", f"{round_syn_future-50}PE"]

In [35]:
scrip_index ,min_val = None , float("inf")

In [38]:
for i in range(3):
    try:
        ce_price = opt[opt.scrip == ce_scrip_list[i]].loc[valid_time[0],'close']
        pe_price = opt[opt.scrip == pe_scrip_list[i]].loc[valid_time[0],'close']
        diff = abs(ce_price - pe_price)
        if min_val> diff:
            min_val = diff
            scrip_index = i 

        print(opt[opt.scrip == ce_scrip_list[scrip_index]].loc[valid_time[0],'close'])
    except:
        pass

55.55
55.55
55.55


In [45]:
sd ,SDroundoff= 1,False

In [47]:
if sd:
    sd_range = (ce_price+pe_price)*sd
    if SDroundoff:
        sd_range = round(sd_range/50)*50
    else:
        sd_range = max(50,round(sd_range/50)*50)

    ce_scrip,pe_scrip = f"{round_fp+sd_range}CE",f"{round_fp+sd_range}PE"
    ce_price,pe_price = opt[opt.scrip == ce_scrip].loc[valid_time[0],'close'] , opt[opt.scrip == pe_scrip].loc[valid_time[0],'close']



get_strangle_strike function

In [56]:
def get_one_om( future_price=None):
    future_price = fut.loc['close'].iloc[0] if future_price is None else future_price
    return int(future_price*50)/100


In [57]:
get_one_om(future_price)

5449.27

In [64]:
opt.head(3)

,index,scrip,open,high,low,close,volume,openinterest,expiry_date,dte
date_time,,,,,,,,,,
2019-02-11 09:15:00,NIFTY,11400CE,0.20,3.00,0.2,0.70,34125,0,2019-02-14,3
2019-02-11 09:15:00,NIFTY,11500CE,0.50,2.40,0.5,0.75,16125,0,2019-02-14,3
2019-02-11 09:15:00,NIFTY,11600CE,2.95,3.05,0.9,0.90,6000,0,2019-02-14,3


In [65]:
current_dt

Timestamp('2019-02-11 09:20:00')

In [72]:
om , target ,tf= 1 , 1,1
one_om = get_one_om(future_price)
target = one_om * om if target is None else target
# opt[(opt.index==current_dt)]
target_od = opt[(opt.index==current_dt)&(opt['close']>=target*tf)].sort_values(by=['close']).copy()


In [ ]:
ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'),'scrip'].iloc[0]
ce_scrip_list = [ce_scrip,f"{int(ce_scrip[:-2])-50}CE",f"{int(ce_scrip[:-2])+50}CE"]
pe_scrip_list = [pe_scrip,f"{int(pe_scrip[:-2])-50}PE",f"{int(pe_scrip[:-2])+50}PE"]

In [76]:
ce_scrip_list

['11150CE', '11100CE', '11200CE']

In [79]:
call_list_prices, put_list_prices = [], []
for z in range(3):
    try:
        call_list_prices.append(opt[opt['scrip'] == ce_scrip_list[z]].loc[current_dt, 'close'])
    except:
        call_list_prices.append(0)
    try:
        put_list_prices.append(opt[opt['scrip'] == pe_scrip_list[z]].loc[current_dt, 'close'])
    except:
        put_list_prices.append(0)


In [80]:
call_list_prices

[np.float64(2.25), np.float64(4.6), np.float64(0.9)]

In [84]:
call,put,min_diff = call_list_prices[0],put_list_prices[0],float('inf')
target_2, target_3 = target*2*tf, target*3


In [85]:
diff = abs(put-call)
required_call, required_put = None, None
if (put+call >= target_2) & (min_diff > diff) & (put+call <= target_3):
    min_diff = diff
    required_call, required_put = call, put


In [86]:
for i in range(1,3):
                    if (min_diff > abs(put_list_prices[i] - call)) & (put_list_prices[i]+call >= target_2) & (put_list_prices[i]+call <= target_3):
                        min_diff = abs(put_list_prices[i] - call)
                        required_call, required_put = call, put_list_prices[i]
                    if (min_diff > abs(call_list_prices[i] - put)) & (call_list_prices[i]+put >= target_2) & (call_list_prices[i]+put <= target_3):
                        min_diff = abs(call_list_prices[i] - put)
                        required_call, required_put = call_list_prices[i], put


In [87]:
ce_scrip, pe_scrip = ce_scrip_list[call_list_prices.index(required_call)], pe_scrip_list[put_list_prices.index(required_put)]
ce_price, pe_price = opt[opt['scrip'] == ce_scrip].loc[current_dt, 'close'], opt[opt['scrip'] == pe_scrip].loc[current_dt, 'close']

In [88]:
ce_scrip,pe_scrip

('11200CE', '10500PE')

In [ ]:
for current_dt in valid_time:
    om , target ,tf= 1 , 1,1
    one_om = get_one_om(future_price)
    target = one_om * om if target is None else target
    # opt[(opt.index==current_dt)]
    target_od = opt[(opt.index==current_dt)&(opt['close']>=target*tf)].sort_values(by=['close']).copy()

    ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
    pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'),'scrip'].iloc[0]
    ce_scrip_list = [ce_scrip,f"{int(ce_scrip[:-2])-50}CE",f"{int(ce_scrip[:-2])+50}CE"]
    pe_scrip_list = [pe_scrip,f"{int(pe_scrip[:-2])-50}PE",f"{int(pe_scrip[:-2])+50}PE"]

    call_list_prices, put_list_prices = [], []
    for z in range(3):
        try:
            call_list_prices.append(opt[opt['scrip'] == ce_scrip_list[z]].loc[current_dt, 'close'])
        except:
            call_list_prices.append(0)
        try:
            put_list_prices.append(opt[opt['scrip'] == pe_scrip_list[z]].loc[current_dt, 'close'])
        except:
            put_list_prices.append(0)
    call,put,min_diff = call_list_prices[0],put_list_prices[0],float('inf')
    target_2, target_3 = target*2*tf, target*3

    diff = abs(put-call)
    required_call, required_put = None, None
    if (put+call >= target_2) & (min_diff > diff) & (put+call <= target_3):
        min_diff = diff
        required_call, required_put = call, put

    for i in range(1,3):
        if (min_diff > abs(put_list_prices[i] - call)) & (put_list_prices[i]+call >= target_2) & (put_list_prices[i]+call <= target_3):
            min_diff = abs(put_list_prices[i] - call)
            required_call, required_put = call, put_list_prices[i]
        if (min_diff > abs(call_list_prices[i] - put)) & (call_list_prices[i]+put >= target_2) & (call_list_prices[i]+put <= target_3):
            min_diff = abs(call_list_prices[i] - put)
            required_call, required_put = call_list_prices[i], put

    ce_scrip, pe_scrip = ce_scrip_list[call_list_prices.index(required_call)], pe_scrip_list[put_list_prices.index(required_put)]
    ce_price, pe_price = opt[opt['scrip'] == ce_scrip].loc[current_dt, 'close'], opt[opt['scrip'] == pe_scrip].loc[current_dt, 'close']
        


        